# 🛡️ Engineering Trustworthy AI: The Decoupled Shield Pattern

**Red Hat Summit Connect 2025**

In [2]:
# Install and import dependencies
!pip install llama-stack-client pandas ipywidgets -q

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from uuid import uuid4
import logging

# Import our demo helpers
from shield_demo_helpers import (
    ShieldMetrics,
    show_hero_banner,
    show_attack_surface,
    show_result_card,
    show_comparison_matrix,
    show_compliance_savings,
    create_interactive_tester,
    TEST_PROMPTS,
    SHIELD_CONFIG
)

# Suppress verbose logging
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("llama_stack_client").setLevel(logging.WARNING)

print("✅ Environment ready")

zsh:1: command not found: pip


ModuleNotFoundError: No module named 'llamastack_client'

In [ ]:
# Show hero banner
show_hero_banner()

## Configuration

In [ ]:
# Configure your deployment
LLAMASTACK_URL = "http://llamastack-trustyai-fms-service.summit-connect-2025.svc.cluster.local:8321"
MODEL_NAME = "tinyllama-1b"

# Connect to LlamaStack
client = LlamaStackClient(base_url=LLAMASTACK_URL)
metrics = ShieldMetrics()

print(f"✅ Connected to {LLAMASTACK_URL}")
print(f"📊 Model: {MODEL_NAME}")

## Part 1: The Trust Gap - Unprotected Agent

In [ ]:
show_attack_surface({'name': 'Unprotected Agent', 'input': False, 'output': False})

In [ ]:
# Demo the trust gap
critical_test = TEST_PROMPTS["pii_multiple"]
print(f"📨 User: {critical_test['prompt'][:80]}...\n")

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": critical_test['prompt']}],
    stream=False
)

show_result_card(
    "PROCESSED WITHOUT VALIDATION",
    "error",
    response.choices[0].message.content[:150] + "...",
    "⚠️ PII was processed without any checks!"
)

## Part 2: Register TrustyAI Shields

In [ ]:
# Register the shield
try:
    client.shields.register(**SHIELD_CONFIG)
    show_result_card("Shield Registered", "allowed", "TrustyAI PII shield is active")
except Exception as e:
    if "already exists" not in str(e).lower():
        raise

## Part 3: Input Shields

In [ ]:
show_attack_surface({'name': 'Input Shield Only', 'input': True, 'output': False})

agent_input = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=['pii_shield'],
    output_shields=[],
    enable_session_persistence=False
)

print("🛡️ Agent created with input shield")

In [ ]:
# Test input shield
session = agent_input.create_session(f"session-{uuid4()}")
response = agent_input.create_turn(
    messages=[{"role": "user", "content": TEST_PROMPTS["pii_multiple"]["prompt"]}],
    session_id=session,
    stream=False
)

# Show result
for step in response.steps:
    if step.step_type == 'shield_call' and step.violation:
        show_result_card("BLOCKED BY INPUT SHIELD", "blocked", step.violation.user_message)
        metrics.record(blocked=True)
        break

## Part 4: Output Shields

In [ ]:
show_attack_surface({'name': 'Output Shield Only', 'input': False, 'output': True})

agent_output = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=[],
    output_shields=['pii_shield'],
    enable_session_persistence=False
)

print("🛡️ Agent created with output shield")

## Part 5: Defense-in-Depth - Both Shields

In [ ]:
show_attack_surface({'name': 'Both Shields (Recommended)', 'input': True, 'output': True})

agent_both = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=['pii_shield'],
    output_shields=['pii_shield'],
    enable_session_persistence=False
)

print("🛡️🛡️ Agent created with maximum protection")

## Part 6: The Complete Picture

In [ ]:
show_comparison_matrix()

## Part 7: Real-World Impact

In [ ]:
show_compliance_savings()

## Part 8: Try It Yourself

In [ ]:
create_interactive_tester(client)

In [ ]:
# Final metrics
metrics.display()